# Búsqueda de texto

Búsqueda de texto en un _corpus_ utilizando la matriz _tf-idf_

La idea es calcular _tf-idf_ para el texto de la búsqueda y encontrar los documentos que tienen mayor valor de la similitud coseno.

$cos(\Theta) = \frac{A \cdot B}{|A| |B|}$

In [1]:
import pandas as pd

In [11]:
tweets = pd.read_feather('../data/tweets_limpios')['text']
tweets

0        promotoresods desear feliz año necesitar esper...
1        aplicación regar aguar depurar enzima natural ...
2        lunes estrenar esperar 19:30 _ 13c viajar para...
3        década cumplir objetivo agenda2030 preparar en...
4        nº 329 homenaje municipalismo 40añosdedemocrac...
                               ...                        
25360    documento cepal examinar tendencia económico s...
25361    ámsterdam demostrar quimera salir entrar crisi...
25362    aprovecha confinamiento blog encontrar 40 artí...
25363    pilar mandato fundación promover debatir ue al...
25364    casa valle imperial proyectar integración tecn...
Name: text, Length: 25365, dtype: object

Construimos la matriz _tf-idf_

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(tweets)
tfidf = vectorizer.transform(tweets)

In [14]:
tfidf.shape

(25365, 25257)

Calculamos el vector _tf-idf_ para el texto de la búsqueda

In [17]:
query = vectorizer.transform(["Luchar por un trabajo digno."])
query.shape

(1, 25257)

Usamos la similitud coseno para encontrar los documentos más parecidos.

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [30]:
%%time

query_result = np.array([cosine_similarity(query, doc) for doc in tfidf]).flatten()

CPU times: user 13.3 s, sys: 5.68 ms, total: 13.3 s
Wall time: 13.3 s


In [31]:
np.argmax(query_result)

11501

In [32]:
query_result[11501]

0.24424456414225454

In [33]:
tweets[11501]

'alineados objetivo agenda2030 proponer _ brindar fuente estable distinto sector población finalidad impulsar crecimiento económico país generar trabajo digno'

In [50]:
result = pd.DataFrame(query_result, columns=['tfidf'])
result_idx = result[result['tfidf'] > 0].sort_values(by='tfidf', ascending=False).index

In [51]:
tweets[result_idx]

11501    alineados objetivo agenda2030 proponer _ brind...
8404     sesión máster ambiente sostenibilidad ods cont...
24833    .por escuchar economía mundo tambalear comprar...
20764    digno aplauso tratar emplear perjudicar rsc so...
20769    digno aplauso tratar emplear perjudicar rsc so...
                               ...                        
15767    acabar bullying acto valentía perseverancia de...
17320    mujeres agua ” campaña latinoamericana 2020 bu...
17389    mujeres agua ” campaña latinoamericana 2020 bu...
9888     participaron josé luis pimentel coordinador ae...
12725    té chai masala alto calidad cápsula tipo nespr...
Name: text, Length: 424, dtype: object

Vamos a probar algunas búsquedas

In [56]:
pd.set_option('display.width', 132)

In [78]:
def search(text):
    query = vectorizer.transform([text])
    query_result = np.array([cosine_similarity(query, doc) for doc in tfidf]).flatten()
    result = pd.DataFrame(query_result, columns=['tfidf'])
    result_idx = result[result['tfidf'] > 0].sort_values(by='tfidf', ascending=False).index
    for idx in result_idx:
        print('- {} [score: {:.4f}]'.format(tweets[idx], result['tfidf'][idx]))

In [79]:
%%time
search('trabajo digno')

- alineados objetivo agenda2030 proponer _ brindar fuente estable distinto sector población finalidad impulsar crecimiento económico país generar trabajo digno [score: 0.4023]
- digno aplauso tratar emplear perjudicar rsc sostenibilidad práctico [score: 0.3173]
- digno aplauso tratar emplear perjudicar rsc sostenibilidad práctico [score: 0.3173]
- agenda2030 verdaderamente agenda gente plan acción acabar pobreza construir vida digno dejar atrás _ conoce ods [score: 0.2892]
- significar vivir condicionar digno garantizar inclusivo sostenible reducir desigualdad convertir ods comprometer cumplir dejar atrás agenda2030 caminar [score: 0.2794]
- significar vivir condicionar digno garantizar inclusivo sostenible reducir desigualdad convertir ods comprometer cumplir dejar atrás agenda2030 caminar [score: 0.2794]
- significar vivir condicionar digno garantizar inclusivo sostenible reducir desigualdad convertir ods comprometer cumplir dejar atrás agenda2030 caminar [score: 0.2794]
- derechoala

In [80]:
%%time
search('Fin de la pobreza')

- pablo iglesias “ patriotismo justicia social medioambiental ” constitucionalismo democratico",de recuperar arts sociales("olvidados)de constitucion agenda2030 [score: 0.3261]
- pablo iglesias “ patriotismo justicia social medioambiental ” constitucionalismo democratico",de recuperar arts sociales("olvidados)de constitucion agenda2030 [score: 0.3261]
- alimentación mesa ¿de conocer consumidor influir hábito alimentario consumidor alimento salud sostenibilidad [score: 0.2348]
- alimentación mesa ¿de conocer consumidor influir hábito alimentario consumidor alimento salud sostenibilidad [score: 0.2348]
- plantar proporcionar 80% alimento generar 98% oxígeno proteger salud clavar eliminar hambre pobreza proteger medioambiente lograr ods 2?de diciembre??lanzamiento año internacional sanidad vegetal,??iyph2020 [score: 0.2326]
- tener leer seriar desarrollar modelar negociar sostenible responsable?la oportunidad negociar empresa interiorizan sostenibilidad responsabilidad sostenibilidad desa

In [81]:
%%time
search('condiciones laborables justas')

- programar mejoramiento condiciones habitabilidad promover configuración territorio equilibrar inclusivo desarrollar infraestructura impulsar acceso equitativo bien espacio público programa mch renovacion urbana ods [score: 0.3143]
CPU times: user 13.6 s, sys: 8.89 ms, total: 13.6 s
Wall time: 13.7 s


In [82]:
%%time
search('alimentos asequibles')

- alimentos desnudo ” campaña eliminar embalaje plástico nomasplastico boicotalplástico nomoreplastic cambio climatico sostenibilidad alimentos desnudos salud [score: 0.4694]
- programa mundial alimentos necesitar 10.000 millón dólar hambre 2020 agenda2030 [score: 0.4030]
- programa mundial alimentos necesitar 10.000 millón dólar hambre 2020 agenda2030 [score: 0.4030]
- programa mundial alimentos necesitar 10.000 millón dólar hambre 2020 agenda2030 [score: 0.4030]
- incluir sostenibilidad vida aprovechar recurso gustar guiar reducir desperdicio alimentos ods descargar [score: 0.3720]
- incluir sostenibilidad vida aprovechar recurso gustar guiar reducir desperdicio alimentos ods descargar [score: 0.3720]
- agenda 2030 desarrollo sostenible ods sistema alimentario deber garantizar alimentación personar futuro alimentos sostenibles resiliencia trabajemos mundo hambre cero feliz2020 [score: 0.3086]
- labor saciar legar familiar hambre conectar cadena evitar desperdiciar sostenibilidad alim